# Functional test with FTDI
- https://www.silabs.com/documents/public/data-sheets/Si5351-B.pdf
- https://www.silabs.com/documents/public/application-notes/AN619.pdf
- https://www.silabs.com/content/usergenerated/asi/cloud/attachments/siliconlabs/en/community/groups/timing/knowledge-base/jcr:content/content/primary/blog/modifying_the_feedba-K8Pv/311668.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '..', '已完成', 'Bridges', 'bitbucket', 'github', 'codes'], 
         ['..', '..', '..', '..', '..', '..', 'Utilities', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline

from clock_generators.si535x.si5351 import Si5351 
from utilities.adapters import peripherals

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
cls = Si5351

cls.DEBUG_MODE_SHOW_BUS_DATA = True         # whether to show bus data. 
cls.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators and Modulators

In [6]:
with_hardware_device = True

if with_hardware_device:
    _i2c = peripherals.I2C.get_Ftdi_i2c() 
    
else:
    _i2c =  None  # using None for testing without actual hardware device.

si = cls(_i2c)  


Action: set_masks 0, writing: 0x0, Address: 2

Action: clear_stickys, writing: 0x0, Address: 1

Action: clear_stickys, writing: 0x0, Address: 15

Action: enable_fanout True, writing: 0x80, Address: 187

Action: Xtal enable_fanout True, writing: 0xc0, Address: 187

Action: set_internal_load_capacitance 10, writing: 0xd2, Address: 183

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 44

Action: set_internal_load_capacitance 10, writing: 0x10, Address: 45

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 46

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 48

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 49

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 42

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 43

Action: set_internal_load_capacitance 


Action: set_frequency 25000000, writing: 0x0, Address: 44

Action: set_frequency 25000000, writing: 0x0, Address: 52

Action: enable_output_clock 1 False, writing: 0x3, Address: 3

Action: enable_output_clock 1 False, writing: 0x43, Address: 17

Action: enable_output_clock 1 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 17

Action: set_frequency 25000000, writing: 0x0, Address: 52

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: enable_output_clock 2 False, writing: 0x7, Address: 3

Action: enable_output_clock 2 False, writing: 0x43, Address: 18

Action: enable_output_clock 2 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 18

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: set_frequency 25000000, writing: 0x0, Address: 68

Action: enable_output_clock 3 False, writing: 0xf, Address: 3

Action: enable_output_clock 3 False, writing: 0x43, Address: 19

Act

## Registers values file I/O

In [7]:
si.map.save_to_file('Initial_values.txt')
initial_values = si.map.read_file('Initial_values.txt') 

In [8]:
freq = 104e5
# freq = 106e5

si.reset()

for i in range(3):
    si.clocks[i].set_frequency(freq)
    
si.print_registers_values()

# print('registers_values = {}'.format(initial_values))


Action: reset, writing: 0x0, Address: 1

Action: reset, writing: 0x0, Address: 2

Action: reset, writing: 0x0, Address: 3

Action: reset, writing: 0x0, Address: 9

Action: reset, writing: 0x0, Address: 15

Action: reset, writing: 0x0, Address: 16

Action: reset, writing: 0x0, Address: 17

Action: reset, writing: 0x0, Address: 18

Action: reset, writing: 0x0, Address: 19

Action: reset, writing: 0x0, Address: 20

Action: reset, writing: 0x0, Address: 21

Action: reset, writing: 0x0, Address: 22

Action: reset, writing: 0x0, Address: 23

Action: reset, writing: 0x0, Address: 24

Action: reset, writing: 0x0, Address: 25

Action: reset, writing: 0x0, Address: 26

Action: reset, writing: 0x0, Address: 27

Action: reset, writing: 0x0, Address: 28

Action: reset, writing: 0x0, Address: 29

Action: reset, writing: 0x0, Address: 30

Action: reset, writing: 0x0, Address: 31

Action: reset, writing: 0x0, Address: 32

Action: reset, writing: 0x0, Address: 33

Action: reset, writing: 0x0, Address:

In [9]:
# clock_builder_values = si.map.read_file('Si5351A-RevB-Registers.txt') 
clock_builder_values = si.map.read_file('Si5351A-RevB-Registers with CenterSS 6ns skew.txt')

In [10]:
# si.write_all_registers(initial_values)

In [11]:
si.map.reset()
si.read_all_registers()
si.map.save_to_file('Read_back_values.txt')
read_back_values = si.map.read_file('Read_back_values.txt')


Action: set_frequency 10400000.0, reading: 0x11, Address: 0

Action: set_frequency 10400000.0, reading: 0xf0, Address: 1

Action: set_frequency 10400000.0, reading: 0x0, Address: 2

Action: set_frequency 10400000.0, reading: 0x0, Address: 3

Action: set_frequency 10400000.0, reading: 0x0, Address: 9

Action: set_frequency 10400000.0, reading: 0x0, Address: 15

Action: set_frequency 10400000.0, reading: 0x0, Address: 16

Action: set_frequency 10400000.0, reading: 0x0, Address: 17

Action: set_frequency 10400000.0, reading: 0x0, Address: 18

Action: set_frequency 10400000.0, reading: 0x0, Address: 19

Action: set_frequency 10400000.0, reading: 0x0, Address: 20

Action: set_frequency 10400000.0, reading: 0x0, Address: 21

Action: set_frequency 10400000.0, reading: 0x0, Address: 22

Action: set_frequency 10400000.0, reading: 0x0, Address: 23

Action: set_frequency 10400000.0, reading: 0x0, Address: 24

Action: set_frequency 10400000.0, reading: 0x0, Address: 25

Action: set_frequency 1040

## Compare Register Value Sets

In [12]:
si.map.compare_values_sets(clock_builder_values, initial_values)

array([[  0.,  nan,   0.,   1.],
       [  1.,  nan,   0.,   1.],
       [  2.,  19.,   0.,   1.],
       [  3.,   0., 248.,   1.],
       [  4.,   0.,  nan,   1.],
       [  7.,   1.,  nan,   1.],
       [  9.,  nan,   0.,   1.],
       [ 15.,   0.,   0.,   0.],
       [ 16.,  15.,  79.,   1.],
       [ 17.,  47.,  79.,   1.],
       [ 18.,  47.,  79.,   1.],
       [ 19., 140., 207.,   1.],
       [ 20., 140., 207.,   1.],
       [ 21., 140., 207.,   1.],
       [ 22., 140., 207.,   1.],
       [ 23., 140., 207.,   1.],
       [ 24.,  nan,   0.,   1.],
       [ 25.,  nan,   0.,   1.],
       [ 26.,   0., 255.,   1.],
       [ 27.,   1., 255.,   1.],
       [ 28.,   0.,   0.,   0.],
       [ 29.,  16.,  16.,   0.],
       [ 30.,   0.,   0.,   0.],
       [ 31.,   0., 240.,   1.],
       [ 32.,   0.,   0.,   0.],
       [ 33.,   0.,   0.,   0.],
       [ 34.,   0., 255.,   1.],
       [ 35.,   1., 255.,   1.],
       [ 36.,   0.,   0.,   0.],
       [ 37.,  16.,  16.,   0.],
       [ 3

In [13]:
si.map.compare_values_sets(initial_values, read_back_values) 

array([[  0.,   0.,  17.,   1.],
       [  1.,   0., 240.,   1.],
       [  2.,   0.,   0.,   0.],
       [  3., 248.,   0.,   1.],
       [  9.,   0.,   0.,   0.],
       [ 15.,   0.,   0.,   0.],
       [ 16.,  79.,   0.,   1.],
       [ 17.,  79.,   0.,   1.],
       [ 18.,  79.,   0.,   1.],
       [ 19., 207.,   0.,   1.],
       [ 20., 207.,   0.,   1.],
       [ 21., 207.,   0.,   1.],
       [ 22., 207.,   0.,   1.],
       [ 23., 207.,   0.,   1.],
       [ 24.,   0.,   0.,   0.],
       [ 25.,   0.,   0.,   0.],
       [ 26., 255.,   0.,   1.],
       [ 27., 255.,   0.,   1.],
       [ 28.,   0.,   0.,   0.],
       [ 29.,  16.,   0.,   1.],
       [ 30.,   0.,   0.,   0.],
       [ 31., 240.,   0.,   1.],
       [ 32.,   0.,   0.,   0.],
       [ 33.,   0.,   0.,   0.],
       [ 34., 255.,   0.,   1.],
       [ 35., 255.,   0.,   1.],
       [ 36.,   0.,   0.,   0.],
       [ 37.,  16.,   0.,   1.],
       [ 38.,   0.,   0.,   0.],
       [ 39., 240.,   0.,   1.],
       [ 4

## Find Integer Dividers for a Frequency

In [14]:
matchs = si.find_integer_dividers(freq_desired = 90e6, even_only = True, torance_hz = 1) 
matchs


Action: Xtal enable_fanout True, writing: 0x40, Address: 187

Action: set_internal_load_capacitance 10, writing: 0xd2, Address: 183

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 44

Action: set_internal_load_capacitance 10, writing: 0x10, Address: 45

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 46

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 48

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 49

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 42

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 43

Action: set_internal_load_capacitance 10, writing: 0x40, Address: 22

Action: reset pll 0, writing: 0x20, Address: 177

Action: pll init., writing: 0x0, Address: 28

Action: pll init., writing: 0x10, Address: 29

Action: pll init., writing:

[((36, 10, 1), (25000000, 900000000, 90000000, 90000000))]

In [15]:
matchs = si.find_integer_dividers(freq_desired = 150e6, even_only = True, torance_hz = 1) 
matchs


Action: Xtal enable_fanout True, writing: 0x50, Address: 187

Action: set_internal_load_capacitance 10, writing: 0xd2, Address: 183

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 44

Action: set_internal_load_capacitance 10, writing: 0x10, Address: 45

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 46

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 48

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 49

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 42

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 43

Action: set_internal_load_capacitance 10, writing: 0x40, Address: 22

Action: reset pll 0, writing: 0x20, Address: 177

Action: pll init., writing: 0x0, Address: 28

Action: pll init., writing: 0x10, Address: 29

Action: pll init., writing:

[((36, 6, 1), (25000000, 900000000, 150000000, 150000000))]

## Find Common Integer Dividers for Frequencies

In [16]:
desired_clock_freqs = (45e6, 30e5, 90e6)
# desired_clock_freqs = (125e6, 48e6, 28.322e6, 74.25e6, 74.25e6, 24.576e6, 22.5792e6, 33.3333e6)
# desired_clock_freqs = (125e6, 48e6, 28.322e6, 74.25e6, 74.25e6, 24.576e6, 22.5792e6, 33.3333e6)
# desired_clock_freqs = (25e6, 25e6)

common_pll_dividers, freqs_pll_dividers, freqs_matches = si.find_integer_pll_dividers_for_clocks(desired_clock_freqs)
common_pll_dividers, freqs_pll_dividers


Action: Xtal enable_fanout True, writing: 0x50, Address: 187

Action: set_internal_load_capacitance 10, writing: 0xd2, Address: 183

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 44

Action: set_internal_load_capacitance 10, writing: 0x10, Address: 45

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 46

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 48

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 49

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 42

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 43

Action: set_internal_load_capacitance 10, writing: 0x40, Address: 22

Action: reset pll 0, writing: 0x20, Address: 177

Action: pll init., writing: 0x0, Address: 28

Action: pll init., writing: 0x10, Address: 29

Action: pll init., writing:

({36}, [{36}, {24, 30, 36}, {36}])

## Main functions test

In [17]:
for f in dir(cls):
    if not f.startswith('_'):
        print('si.{}()'.format(f))

si.DEBUG_MODE()
si.DEBUG_MODE_PRINT_REGISTER()
si.DEBUG_MODE_SHOW_BUS_DATA()
si.FREQ_REF()
si.HAS_VCXO()
si.I2C_ADDRESS()
si.N_OUTPUT_CLOCKS()
si.OUTPUT_CLOCKS_IN_USE()
si.READ_ONLY_REGISTERS()
si.close()
si.current_frequency()
si.current_phase()
si.do()
si.do_on_devices()
si.enable()
si.enable_output()
si.enable_output_channel()
si.find_integer_dividers()
si.find_integer_pll_dividers_for_clocks()
si.init()
si.is_virtual_device()
si.load_registers()
si.pause()
si.print()
si.print_registers_values()
si.read_all_registers()
si.ready()
si.reset()
si.reset_plls()
si.restore_clocks_freqs()
si.resume()
si.revision()
si.set_frequency()
si.set_phase()
si.start()
si.status()
si.stop()
si.update()
si.write_all_registers()


In [18]:
si.init()


Action: set_masks 0, writing: 0x0, Address: 2

Action: clear_stickys, writing: 0x0, Address: 1

Action: clear_stickys, writing: 0x0, Address: 15

Action: enable_fanout True, writing: 0xd0, Address: 187

Action: Xtal enable_fanout True, writing: 0xd0, Address: 187

Action: set_internal_load_capacitance 10, writing: 0xd2, Address: 183

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 44

Action: set_internal_load_capacitance 10, writing: 0x10, Address: 45

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 46

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 48

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 49

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 42

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 43

Action: set_internal_load_capacitance

Action: set_input_source MultiSynth, writing: 0x4f, Address: 17

Action: set_frequency 25000000, writing: 0x0, Address: 52

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: enable_output_clock 2 False, writing: 0x7, Address: 3

Action: enable_output_clock 2 False, writing: 0x43, Address: 18

Action: enable_output_clock 2 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 18

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: set_frequency 25000000, writing: 0x0, Address: 68

Action: enable_output_clock 3 False, writing: 0xf, Address: 3

Action: enable_output_clock 3 False, writing: 0x43, Address: 19

Action: enable_output_clock 3 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 19

Action: set_frequency 25000000, writing: 0x0, Address: 68

Action: set_frequency 25000000, writing: 0x0, Address: 76

Action: enable_output_clock 4 False, writing: 0x1f, Address: 3

Act

In [ ]:
init_actions = '''

Action: set_masks 0, writing: 0x0, Address: 2

Action: clear_stickys, writing: 0x0, Address: 1

Action: clear_stickys, writing: 0x0, Address: 15

Action: enable_fanout True, writing: 0xd0, Address: 187

Action: Xtal enable_fanout True, writing: 0xd0, Address: 187

Action: set_internal_load_capacitance 10, writing: 0xd2, Address: 183

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 44

Action: set_internal_load_capacitance 10, writing: 0x10, Address: 45

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 46

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 48

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 49

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 42

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 43

Action: set_internal_load_capacitance 10, writing: 0x40, Address: 22

Action: reset pll 0, writing: 0x20, Address: 177

Action: pll init., writing: 0x0, Address: 28

Action: pll init., writing: 0x10, Address: 29

Action: pll init., writing: 0x0, Address: 30

Action: pll init., writing: 0xf0, Address: 31

Action: pll init., writing: 0x0, Address: 32

Action: pll init., writing: 0x0, Address: 33

Action: pll init., writing: 0xf0, Address: 31

Action: pll init., writing: 0xff, Address: 26

Action: pll init., writing: 0xff, Address: 27

Action: pll init., writing: 0x40, Address: 22

Action: reset pll 0, writing: 0x20, Address: 177

Action: pll set_input_source xtal_as_source = True, writing: 0x0, Address: 15

Action: reset pll 0, writing: 0x20, Address: 177

Action: reset pll 0, writing: 0x0, Address: 52

Action: reset pll 0, writing: 0x10, Address: 53

Action: reset pll 0, writing: 0x0, Address: 54

Action: reset pll 0, writing: 0xf0, Address: 55

Action: reset pll 0, writing: 0x0, Address: 56

Action: reset pll 0, writing: 0x0, Address: 57

Action: reset pll 0, writing: 0xf0, Address: 55

Action: reset pll 0, writing: 0xff, Address: 50

Action: reset pll 0, writing: 0xff, Address: 51

Action: reset pll 0, writing: 0x40, Address: 23

Action: reset pll 1, writing: 0x80, Address: 177

Action: pll init., writing: 0x0, Address: 36

Action: pll init., writing: 0x10, Address: 37

Action: pll init., writing: 0x0, Address: 38

Action: pll init., writing: 0x0, Address: 39

Action: pll init., writing: 0x0, Address: 40

Action: pll init., writing: 0x0, Address: 41

Action: pll init., writing: 0xf0, Address: 39

Action: pll init., writing: 0xff, Address: 34

Action: pll init., writing: 0xff, Address: 35

Action: pll init., writing: 0x40, Address: 23

Action: reset pll 1, writing: 0x80, Address: 177

Action: pll set_input_source xtal_as_source = True, writing: 0x0, Address: 15

Action: reset pll 1, writing: 0x80, Address: 177

Action: reset pll 1, writing: 0x0, Address: 44

Action: reset pll 1, writing: 0x10, Address: 45

Action: reset pll 1, writing: 0x0, Address: 46

Action: reset pll 1, writing: 0xf0, Address: 47

Action: reset pll 1, writing: 0x0, Address: 48

Action: reset pll 1, writing: 0x0, Address: 49

Action: reset pll 1, writing: 0xf0, Address: 47

Action: reset pll 1, writing: 0xff, Address: 42

Action: reset pll 1, writing: 0xff, Address: 43

Action: reset pll 1, writing: 0x4f, Address: 16

Action: multisynth set_input_source 0, writing: 0x4f, Address: 16

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x0, Address: 52

Action: multisynth enable_fanout True, writing: 0x10, Address: 53

Action: multisynth enable_fanout True, writing: 0x0, Address: 54

Action: multisynth enable_fanout True, writing: 0xf0, Address: 55

Action: multisynth enable_fanout True, writing: 0x0, Address: 56

Action: multisynth enable_fanout True, writing: 0x0, Address: 57

Action: multisynth enable_fanout True, writing: 0xf0, Address: 55

Action: multisynth enable_fanout True, writing: 0xff, Address: 50

Action: multisynth enable_fanout True, writing: 0xff, Address: 51

Action: multisynth enable_fanout True, writing: 0x40, Address: 17

Action: multisynth set_input_source 0, writing: 0x40, Address: 17

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x0, Address: 60

Action: multisynth enable_fanout True, writing: 0x10, Address: 61

Action: multisynth enable_fanout True, writing: 0x0, Address: 62

Action: multisynth enable_fanout True, writing: 0xf0, Address: 63

Action: multisynth enable_fanout True, writing: 0x0, Address: 64

Action: multisynth enable_fanout True, writing: 0x0, Address: 65

Action: multisynth enable_fanout True, writing: 0xf0, Address: 63

Action: multisynth enable_fanout True, writing: 0xff, Address: 58

Action: multisynth enable_fanout True, writing: 0xff, Address: 59

Action: multisynth enable_fanout True, writing: 0x40, Address: 18

Action: multisynth set_input_source 0, writing: 0x40, Address: 18

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x0, Address: 68

Action: multisynth enable_fanout True, writing: 0x10, Address: 69

Action: multisynth enable_fanout True, writing: 0x0, Address: 70

Action: multisynth enable_fanout True, writing: 0x0, Address: 71

Action: multisynth enable_fanout True, writing: 0x0, Address: 72

Action: multisynth enable_fanout True, writing: 0x0, Address: 73

Action: multisynth enable_fanout True, writing: 0xf0, Address: 71

Action: multisynth enable_fanout True, writing: 0xff, Address: 66

Action: multisynth enable_fanout True, writing: 0xff, Address: 67

Action: multisynth enable_fanout True, writing: 0x40, Address: 19

Action: multisynth set_input_source 0, writing: 0x40, Address: 19

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x0, Address: 76

Action: multisynth enable_fanout True, writing: 0x10, Address: 77

Action: multisynth enable_fanout True, writing: 0x0, Address: 78

Action: multisynth enable_fanout True, writing: 0x0, Address: 79

Action: multisynth enable_fanout True, writing: 0x0, Address: 80

Action: multisynth enable_fanout True, writing: 0x0, Address: 81

Action: multisynth enable_fanout True, writing: 0xf0, Address: 79

Action: multisynth enable_fanout True, writing: 0xff, Address: 74

Action: multisynth enable_fanout True, writing: 0xff, Address: 75

Action: multisynth enable_fanout True, writing: 0x40, Address: 20

Action: multisynth set_input_source 0, writing: 0x40, Address: 20

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x0, Address: 84

Action: multisynth enable_fanout True, writing: 0x10, Address: 85

Action: multisynth enable_fanout True, writing: 0x0, Address: 86

Action: multisynth enable_fanout True, writing: 0x0, Address: 87

Action: multisynth enable_fanout True, writing: 0x0, Address: 88

Action: multisynth enable_fanout True, writing: 0x0, Address: 89

Action: multisynth enable_fanout True, writing: 0xf0, Address: 87

Action: multisynth enable_fanout True, writing: 0xff, Address: 82

Action: multisynth enable_fanout True, writing: 0xff, Address: 83

Action: multisynth enable_fanout True, writing: 0x40, Address: 21

Action: multisynth set_input_source 0, writing: 0x40, Address: 21

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x24, Address: 90

Action: multisynth set_input_source 0, writing: 0x40, Address: 22

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x24, Address: 91

Action: multisynth set_input_source 0, writing: 0x40, Address: 23

Action: multisynth enable_fanout True, writing: 0xd0, Address: 187

Action: multisynth enable_fanout True, writing: 0x0, Address: 44

Action: enable_output_clock 0 False, writing: 0x1, Address: 3

Action: enable_output_clock 0 False, writing: 0x4f, Address: 16

Action: enable_output_clock 0 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 16

Action: set_frequency 25000000, writing: 0x0, Address: 44

Action: set_frequency 25000000, writing: 0x0, Address: 52

Action: enable_output_clock 1 False, writing: 0x3, Address: 3

Action: enable_output_clock 1 False, writing: 0x43, Address: 17

Action: enable_output_clock 1 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 17

Action: set_frequency 25000000, writing: 0x0, Address: 52

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: enable_output_clock 2 False, writing: 0x7, Address: 3

Action: enable_output_clock 2 False, writing: 0x43, Address: 18

Action: enable_output_clock 2 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 18

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: set_frequency 25000000, writing: 0x0, Address: 68

Action: enable_output_clock 3 False, writing: 0xf, Address: 3

Action: enable_output_clock 3 False, writing: 0x43, Address: 19

Action: enable_output_clock 3 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 19

Action: set_frequency 25000000, writing: 0x0, Address: 68

Action: set_frequency 25000000, writing: 0x0, Address: 76

Action: enable_output_clock 4 False, writing: 0x1f, Address: 3

Action: enable_output_clock 4 False, writing: 0x43, Address: 20

Action: enable_output_clock 4 False, writing: 0x0, Address: 25

Action: set_input_source MultiSynth, writing: 0x4f, Address: 20

Action: set_frequency 25000000, writing: 0x0, Address: 76

Action: set_frequency 25000000, writing: 0x0, Address: 84

Action: enable_output_clock 5 False, writing: 0x3f, Address: 3

Action: enable_output_clock 5 False, writing: 0x43, Address: 21

Action: enable_output_clock 5 False, writing: 0x0, Address: 25

Action: set_input_source MultiSynth, writing: 0x4f, Address: 21

Action: set_frequency 25000000, writing: 0x0, Address: 84

Action: set_frequency 25000000, writing: 0x0, Address: 92

Action: enable_output_clock 6 False, writing: 0x7f, Address: 3

Action: enable_output_clock 6 False, writing: 0x43, Address: 22

Action: enable_output_clock 6 False, writing: 0x0, Address: 25

Action: set_input_source MultiSynth, writing: 0x4f, Address: 22

Action: set_frequency 25000000, writing: 0x0, Address: 92

Action: set_frequency 25000000, writing: 0x0, Address: 92

Action: enable_output_clock 7 False, writing: 0xff, Address: 3

Action: enable_output_clock 7 False, writing: 0x43, Address: 23

Action: enable_output_clock 7 False, writing: 0x0, Address: 25

Action: set_input_source MultiSynth, writing: 0x4f, Address: 23

Action: set_frequency 25000000, writing: 0x0, Address: 92

Action: enable Spread Spectrum False, writing: 0x0, Address: 149

Action: enable Spread Spectrum False, writing: 0x0, Address: 149

Action: reset pll 0, writing: 0x20, Address: 177

Action: enable_output_clock 0 False, writing: 0xff, Address: 3

Action: enable_output_clock 0 False, writing: 0xcf, Address: 16

Action: enable_output_clock 1 False, writing: 0xff, Address: 3

Action: enable_output_clock 1 False, writing: 0xcf, Address: 17

Action: enable_output_clock 2 False, writing: 0xff, Address: 3

Action: enable_output_clock 2 False, writing: 0xcf, Address: 18

Action: enable_output_clock 3 False, writing: 0xff, Address: 3

Action: enable_output_clock 3 False, writing: 0xcf, Address: 19

Action: enable_output_clock 4 False, writing: 0xff, Address: 3

Action: enable_output_clock 4 False, writing: 0xcf, Address: 20

Action: enable_output_clock 5 False, writing: 0xff, Address: 3

Action: enable_output_clock 5 False, writing: 0xcf, Address: 21

Action: enable_output_clock 6 False, writing: 0xff, Address: 3

Action: enable_output_clock 6 False, writing: 0xcf, Address: 22

Action: enable_output_clock 7 False, writing: 0xff, Address: 3

Action: enable_output_clock 7 False, writing: 0xcf, Address: 23

Action: power_down False, writing: 0x4f, Address: 16

Action: power_down False, writing: 0x4f, Address: 17

Action: power_down False, writing: 0x4f, Address: 18

Action: reset_plls, writing: 0xa0, Address: 177

Action: enable_output_clock 0 True, writing: 0xfe, Address: 3

Action: enable_output_clock 1 True, writing: 0xfc, Address: 3

Action: enable_output_clock 2 True, writing: 0xf8, Address: 3
'''

In [19]:
si.is_virtual_device

False

In [20]:
si.revision


Action: enable_output_clock 2 True, reading: 0x11, Address: 0


1

In [21]:
# si.set_frequency(2323)

In [22]:
# si.set_phase(55)

In [23]:
si.reset()


Action: reset, writing: 0x0, Address: 1

Action: reset, writing: 0x0, Address: 2

Action: reset, writing: 0x0, Address: 3

Action: reset, writing: 0x0, Address: 9

Action: reset, writing: 0x0, Address: 15

Action: reset, writing: 0x0, Address: 16

Action: reset, writing: 0x0, Address: 17

Action: reset, writing: 0x0, Address: 18

Action: reset, writing: 0x0, Address: 19

Action: reset, writing: 0x0, Address: 20

Action: reset, writing: 0x0, Address: 21

Action: reset, writing: 0x0, Address: 22

Action: reset, writing: 0x0, Address: 23

Action: reset, writing: 0x0, Address: 24

Action: reset, writing: 0x0, Address: 25

Action: reset, writing: 0x0, Address: 26

Action: reset, writing: 0x0, Address: 27

Action: reset, writing: 0x0, Address: 28

Action: reset, writing: 0x0, Address: 29

Action: reset, writing: 0x0, Address: 30

Action: reset, writing: 0x0, Address: 31

Action: reset, writing: 0x0, Address: 32

Action: reset, writing: 0x0, Address: 33

Action: reset, writing: 0x0, Address:

In [24]:
# si.current_frequency

In [25]:
# si.current_phase

In [26]:
si.enable()


Action: enable_output_clock 0 True, writing: 0x0, Address: 3

Action: enable_output_clock 1 True, writing: 0x0, Address: 3

Action: enable_output_clock 2 True, writing: 0x0, Address: 3

Action: enable_output_clock 3 True, writing: 0x0, Address: 3

Action: enable_output_clock 4 True, writing: 0x0, Address: 3

Action: enable_output_clock 5 True, writing: 0x0, Address: 3

Action: enable_output_clock 6 True, writing: 0x0, Address: 3

Action: enable_output_clock 7 True, writing: 0x0, Address: 3


In [27]:
si.enable_output()


Action: enable_output_clock 0 True, writing: 0x0, Address: 3

Action: enable_output_clock 1 True, writing: 0x0, Address: 3

Action: enable_output_clock 2 True, writing: 0x0, Address: 3

Action: enable_output_clock 3 True, writing: 0x0, Address: 3

Action: enable_output_clock 4 True, writing: 0x0, Address: 3

Action: enable_output_clock 5 True, writing: 0x0, Address: 3

Action: enable_output_clock 6 True, writing: 0x0, Address: 3

Action: enable_output_clock 7 True, writing: 0x0, Address: 3


In [28]:
si.enable_output_channel(0)


Action: enable_output_clock 0 True, writing: 0x0, Address: 3


In [29]:
si.load_registers(initial_values)

In [30]:
si.read_all_registers()


Action: load_registers_values, reading: 0x11, Address: 0

Action: load_registers_values, reading: 0xf0, Address: 1

Action: load_registers_values, reading: 0x0, Address: 2

Action: load_registers_values, reading: 0x0, Address: 3

Action: load_registers_values, reading: 0x0, Address: 9

Action: load_registers_values, reading: 0x0, Address: 15

Action: load_registers_values, reading: 0x0, Address: 16

Action: load_registers_values, reading: 0x0, Address: 17

Action: load_registers_values, reading: 0x0, Address: 18

Action: load_registers_values, reading: 0x0, Address: 19

Action: load_registers_values, reading: 0x0, Address: 20

Action: load_registers_values, reading: 0x0, Address: 21

Action: load_registers_values, reading: 0x0, Address: 22

Action: load_registers_values, reading: 0x0, Address: 23

Action: load_registers_values, reading: 0x0, Address: 24

Action: load_registers_values, reading: 0x0, Address: 25

Action: load_registers_values, reading: 0x0, Address: 26

Action: load_reg

[(0, 'Device_Status', 17),
 (1, 'Interrupt_Status_Sticky', 240),
 (2, 'Interrupt_Status_Mask', 0),
 (3, 'Output_Enable_Control', 0),
 (9, 'OEB_Pin_Enable_Control_Mask', 0),
 (15, 'PLL_Input_Source', 0),
 (16, 'CLK0_Control', 0),
 (17, 'CLK1_Control', 0),
 (18, 'CLK2_Control', 0),
 (19, 'CLK3_Control', 0),
 (20, 'CLK4_Control', 0),
 (21, 'CLK5_Control', 0),
 (22, 'CLK6_Control', 0),
 (23, 'CLK7_Control', 0),
 (24, 'CLK3_0_Disable_State', 0),
 (25, 'CLK7_4_Disable_State', 0),
 (26, 'Multisynth_NA_Parameters_26', 0),
 (27, 'Multisynth_NA_Parameters_27', 0),
 (28, 'Multisynth_NA_Parameters_28', 0),
 (29, 'Multisynth_NA_Parameters_29', 0),
 (30, 'Multisynth_NA_Parameters_30', 0),
 (31, 'Multisynth_NA_Parameters_31', 0),
 (32, 'Multisynth_NA_Parameters_32', 0),
 (33, 'Multisynth_NA_Parameters_33', 0),
 (34, 'Multisynth_NB_Parameters_34', 0),
 (35, 'Multisynth_NB_Parameters_35', 0),
 (36, 'Multisynth_NB_Parameters_36', 0),
 (37, 'Multisynth_NB_Parameters_37', 0),
 (38, 'Multisynth_NB_Paramete

In [31]:
si.ready


Action: load_registers_values, reading: 0x11, Address: 0


True

In [32]:
si.status.print()


Action: load_registers_values, reading: 0x11, Address: 0

<< Device_Status >>:  ('0x11', '0b10001')
[ SYS_INIT ]   :  0
[ LOL_B ]      :  0
[ LOL_A ]      :  0
[ LOS_CLKIN ]  :  1
[ LOS_XTAL ]   :  0
[ Reserved_2 ] :  0
[ REVID ]      :  1


10

In [33]:
si.restore_clocks_freqs()


Action: set_frequency 25000000, writing: 0x0, Address: 44

Action: set_frequency 25000000, writing: 0x0, Address: 52

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: set_frequency 25000000, writing: 0x0, Address: 68

Action: set_frequency 25000000, writing: 0x0, Address: 76

Action: set_frequency 25000000, writing: 0x0, Address: 84

Action: set_frequency 25000000, writing: 0x0, Address: 92

Action: set_frequency 25000000, writing: 0x0, Address: 92


In [34]:
si.write_all_registers()


Action: set_frequency 25000000, writing: 0xf0, Address: 1

Action: set_frequency 25000000, writing: 0x0, Address: 2

Action: set_frequency 25000000, writing: 0x0, Address: 3

Action: set_frequency 25000000, writing: 0x0, Address: 9

Action: set_frequency 25000000, writing: 0x0, Address: 15

Action: set_frequency 25000000, writing: 0x0, Address: 16

Action: set_frequency 25000000, writing: 0x0, Address: 17

Action: set_frequency 25000000, writing: 0x0, Address: 18

Action: set_frequency 25000000, writing: 0x0, Address: 19

Action: set_frequency 25000000, writing: 0x0, Address: 20

Action: set_frequency 25000000, writing: 0x0, Address: 21

Action: set_frequency 25000000, writing: 0x0, Address: 22

Action: set_frequency 25000000, writing: 0x0, Address: 23

Action: set_frequency 25000000, writing: 0x0, Address: 24

Action: set_frequency 25000000, writing: 0x0, Address: 25

Action: set_frequency 25000000, writing: 0x0, Address: 26

Action: set_frequency 25000000, writing: 0x0, Address: 27


In [35]:
si.start()


Action: enable_output_clock 0 True, writing: 0x0, Address: 3

Action: enable_output_clock 1 True, writing: 0x0, Address: 3

Action: enable_output_clock 2 True, writing: 0x0, Address: 3

Action: enable_output_clock 3 True, writing: 0x0, Address: 3

Action: enable_output_clock 4 True, writing: 0x0, Address: 3

Action: enable_output_clock 5 True, writing: 0x0, Address: 3

Action: enable_output_clock 6 True, writing: 0x0, Address: 3

Action: enable_output_clock 7 True, writing: 0x0, Address: 3


In [36]:
si.pause()


Action: enable_output_clock 0 False, writing: 0x1, Address: 3

Action: enable_output_clock 1 False, writing: 0x3, Address: 3

Action: enable_output_clock 2 False, writing: 0x7, Address: 3

Action: enable_output_clock 3 False, writing: 0xf, Address: 3

Action: enable_output_clock 4 False, writing: 0x1f, Address: 3

Action: enable_output_clock 5 False, writing: 0x3f, Address: 3

Action: enable_output_clock 6 False, writing: 0x7f, Address: 3

Action: enable_output_clock 7 False, writing: 0xff, Address: 3


In [37]:
si.resume()


Action: enable_output_clock 0 True, writing: 0xfe, Address: 3

Action: enable_output_clock 1 True, writing: 0xfc, Address: 3

Action: enable_output_clock 2 True, writing: 0xf8, Address: 3

Action: enable_output_clock 3 True, writing: 0xf0, Address: 3

Action: enable_output_clock 4 True, writing: 0xe0, Address: 3

Action: enable_output_clock 5 True, writing: 0xc0, Address: 3

Action: enable_output_clock 6 True, writing: 0x80, Address: 3

Action: enable_output_clock 7 True, writing: 0x0, Address: 3


In [38]:
si.update()


Action: update, writing: 0xf0, Address: 1

Action: update, writing: 0x0, Address: 2

Action: update, writing: 0x0, Address: 3

Action: update, writing: 0x0, Address: 9

Action: update, writing: 0x0, Address: 15

Action: update, writing: 0x0, Address: 16

Action: update, writing: 0x0, Address: 17

Action: update, writing: 0x0, Address: 18

Action: update, writing: 0x0, Address: 19

Action: update, writing: 0x0, Address: 20

Action: update, writing: 0x0, Address: 21

Action: update, writing: 0x0, Address: 22

Action: update, writing: 0x0, Address: 23

Action: update, writing: 0x0, Address: 24

Action: update, writing: 0x0, Address: 25

Action: update, writing: 0x0, Address: 26

Action: update, writing: 0x0, Address: 27

Action: update, writing: 0x0, Address: 28

Action: update, writing: 0x0, Address: 29

Action: update, writing: 0x0, Address: 30

Action: update, writing: 0x0, Address: 31

Action: update, writing: 0x0, Address: 32

Action: update, writing: 0x0, Address: 33

Action: updat

In [39]:
si.stop()


Action: enable_output_clock 0 False, writing: 0x1, Address: 3

Action: enable_output_clock 1 False, writing: 0x3, Address: 3

Action: enable_output_clock 2 False, writing: 0x7, Address: 3

Action: enable_output_clock 3 False, writing: 0xf, Address: 3

Action: enable_output_clock 4 False, writing: 0x1f, Address: 3

Action: enable_output_clock 5 False, writing: 0x3f, Address: 3

Action: enable_output_clock 6 False, writing: 0x7f, Address: 3

Action: enable_output_clock 7 False, writing: 0xff, Address: 3


In [40]:
si.close()


Action: enable_output_clock 0 False, writing: 0xff, Address: 3

Action: enable_output_clock 1 False, writing: 0xff, Address: 3

Action: enable_output_clock 2 False, writing: 0xff, Address: 3

Action: enable_output_clock 3 False, writing: 0xff, Address: 3

Action: enable_output_clock 4 False, writing: 0xff, Address: 3

Action: enable_output_clock 5 False, writing: 0xff, Address: 3

Action: enable_output_clock 6 False, writing: 0xff, Address: 3

Action: enable_output_clock 7 False, writing: 0xff, Address: 3


In [41]:
si.print()


<< Device_Status >>:  ('0x11', '0b10001')
[ SYS_INIT ]   :  0
[ LOL_B ]      :  0
[ LOL_A ]      :  0
[ LOS_CLKIN ]  :  1
[ LOS_XTAL ]   :  0
[ Reserved_2 ] :  0
[ REVID ]      :  1

<< Interrupt_Status_Sticky >>:  ('0xf0', '0b11110000')
[ SYS_INIT_STKY ]  :  1
[ LOL_B_STKY ]     :  1
[ LOL_A_STKY ]     :  1
[ LOS_CLKIN_STKY ] :  1
[ LOS_XTAL_STKY ]  :  0
[ Reserved_0 ]     :  0

<< Interrupt_Status_Mask >>:  ('0x0', '0b0')
[ SYS_INIT_MASK ]   :  0
[ LOL_B_MASK ]      :  0
[ LOL_A_MASK ]      :  0
[ LOS__CLKIN_MASK ] :  0
[ LOS__XTAL_MASK ]  :  0
[ Reserved_0 ]      :  0

<< Output_Enable_Control >>:  ('0xff', '0b11111111')
[ CLK7_OEB ] :  1
[ CLK6_OEB ] :  1
[ CLK5_OEB ] :  1
[ CLK4_OEB ] :  1
[ CLK3_OEB ] :  1
[ CLK2_OEB ] :  1
[ CLK1_OEB ] :  1
[ CLK0_OEB ] :  1

<< OEB_Pin_Enable_Control_Mask >>:  ('0x0', '0b0')
[ OEB_MASK_7 ] :  0
[ OEB_MASK_6 ] :  0
[ OEB_MASK_5 ] :  0
[ OEB_MASK_4 ] :  0
[ OEB_MASK_3 ] :  0
[ OEB_MASK_2 ] :  0
[ OEB_MASK_1 ] :  0
[ OEB_MASK_0 ] :  0

<< PLL_Inp

## Spread Spectrum test

In [42]:
si.init()


Action: set_masks 0, writing: 0x0, Address: 2

Action: clear_stickys, writing: 0x0, Address: 1

Action: clear_stickys, writing: 0x0, Address: 15

Action: enable_fanout True, writing: 0x80, Address: 187

Action: Xtal enable_fanout True, writing: 0xc0, Address: 187

Action: set_internal_load_capacitance 10, writing: 0xd2, Address: 183

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 44

Action: set_internal_load_capacitance 10, writing: 0x10, Address: 45

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 46

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 48

Action: set_internal_load_capacitance 10, writing: 0x0, Address: 49

Action: set_internal_load_capacitance 10, writing: 0xf0, Address: 47

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 42

Action: set_internal_load_capacitance 10, writing: 0xff, Address: 43

Action: set_internal_load_capacitance 


Action: enable_output_clock 1 False, writing: 0x43, Address: 17

Action: enable_output_clock 1 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 17

Action: set_frequency 25000000, writing: 0x0, Address: 52

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: enable_output_clock 2 False, writing: 0xff, Address: 3

Action: enable_output_clock 2 False, writing: 0x43, Address: 18

Action: enable_output_clock 2 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 18

Action: set_frequency 25000000, writing: 0x0, Address: 60

Action: set_frequency 25000000, writing: 0x0, Address: 68

Action: enable_output_clock 3 False, writing: 0xff, Address: 3

Action: enable_output_clock 3 False, writing: 0x43, Address: 19

Action: enable_output_clock 3 False, writing: 0x0, Address: 24

Action: set_input_source MultiSynth, writing: 0x4f, Address: 19

Action: set_frequency 25000000, writing: 0x0, Address

In [43]:
si.spread_spectrum.enable(True, mode = 'center', ssc_amp = 0.01)


Action: enable Spread Spectrum True, writing: 0x8f, Address: 22

Action: set_center_spread ssc_amp 0.01, writing: 0x0, Address: 154

Action: set_center_spread ssc_amp 0.01, writing: 0xc6, Address: 155

Action: set_center_spread ssc_amp 0.01, writing: 0x0, Address: 161

Action: set_center_spread ssc_amp 0.01, writing: 0x0, Address: 160

Action: set_center_spread ssc_amp 0.01, writing: 0x1e, Address: 156

Action: set_center_spread ssc_amp 0.01, writing: 0x16, Address: 157

Action: set_center_spread ssc_amp 0.01, writing: 0x7f, Address: 158

Action: set_center_spread ssc_amp 0.01, writing: 0xff, Address: 159

Action: set_center_spread ssc_amp 0.01, writing: 0x0, Address: 154

Action: set_center_spread ssc_amp 0.01, writing: 0x0, Address: 153

Action: set_center_spread ssc_amp 0.01, writing: 0x1d, Address: 149

Action: set_center_spread ssc_amp 0.01, writing: 0x7e, Address: 150

Action: set_center_spread ssc_amp 0.01, writing: 0x7f, Address: 151

Action: set_center_spread ssc_amp 0.01, wr

In [44]:
si.spread_spectrum.enable(True, mode = 'down', ssc_amp = 0.01)


Action: enable Spread Spectrum True, writing: 0x8f, Address: 22

Action: set_down_spread ssc_amp 0.01, writing: 0x0, Address: 154

Action: set_down_spread ssc_amp 0.01, writing: 0xc6, Address: 155

Action: set_down_spread ssc_amp 0.01, writing: 0x0, Address: 161

Action: set_down_spread ssc_amp 0.01, writing: 0x0, Address: 160

Action: set_down_spread ssc_amp 0.01, writing: 0x0, Address: 156

Action: set_down_spread ssc_amp 0.01, writing: 0x0, Address: 157

Action: set_down_spread ssc_amp 0.01, writing: 0x7f, Address: 158

Action: set_down_spread ssc_amp 0.01, writing: 0xff, Address: 159

Action: set_down_spread ssc_amp 0.01, writing: 0x0, Address: 154

Action: set_down_spread ssc_amp 0.01, writing: 0x0, Address: 153

Action: set_down_spread ssc_amp 0.01, writing: 0x8e, Address: 149

Action: set_down_spread ssc_amp 0.01, writing: 0xbf, Address: 150

Action: set_down_spread ssc_amp 0.01, writing: 0xff, Address: 151

Action: set_down_spread ssc_amp 0.01, writing: 0xff, Address: 152

Act

In [45]:
si.spread_spectrum.enable(False)


Action: enable Spread Spectrum False, writing: 0xe, Address: 149

Action: enable Spread Spectrum False, writing: 0xe, Address: 149

Action: reset pll 0, writing: 0x20, Address: 177


In [46]:
clk = si.clocks[0]